# SQL : Group By, Having & Count  
  


## COUNT()  
  
**COUNT()**, returns a count of things. Pass it a the name of a column, it returns the number of entries in that column.  
  


In [ ]:
query = """
        SELECT COUNT(ID)  
        FROM `bigquery-public-data.pet_records.pets`
        """

**COUNT()** is an example of an **aggregate function**, which takes many values and returns one.  

## GROUP BY  
**GROUP BY** takes the name of one or more columns, and treats all rows with the same value in the column as a single group when you apply aggregate functions like **COUNT()**.  
  

In [ ]:
query="""
      SELECT Animal, COUNT(ID)  
      FROM `bigquery-public-data.pet_records.pets`  
      GROUP BY Animal
      """

It returns a table with three rows (one for each distinct animal). 

## GROUP BY ... HAVING  
**HAVING** is used in combination with **GROUP BY** to ignore groups that don't meet certain criteria.  


In [ ]:
query="""
      SELECT Animal COUNT(ID)
      FROM `bigquery-public-data.pet_records.pets`
      GROUP BY Animal  
      HAVING COUNT(ID) > 1
      """

## Example: Which Hacker News comments generated the most discussion?  
Ready to see an example on a real dataset? The Hacker News dataset contains information on stories and comments from the Hacker News social networking site.  
  
We'll work with the comments table and begin by printing the first few rows.

In [ ]:
# Query to select comments that received more than 10 replies
query_popular = """
                SELECT parent, COUNT(id)
                FROM `bigquery-public-data.hacker_news.comments`
                GROUP BY parent
                HAVING COUNT(id) > 10
                """

Now that our query is ready, let's run it and store the results in a pandas DataFrame:


In [ ]:
# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 10 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(query_popular, job_config=safe_config)

# API request - run the query, and convert the results to a pandas DataFrame
popular_comments = query_job.to_dataframe()

# Print the first five rows of the DataFrame
popular_comments.head()

## Aliasing and other improvements  
A couple hints to make your queries even better:  
* The column resulting from **COUNT(id)** was called f0__. That's not a very descriptive name. You can change the name by adding `AS NumPosts` after you specify the aggregation. This is called **aliasing**, and it will be covered in more detail in an upcoming lesson.  
* If you are ever unsure what to put inside the **COUNT()** function, you can do `COUNT(1)` to count the rows in each group. Most people find it especially readable, because we know it's not focusing on other columns. It also scans less data than if supplied column names (making it faster and using less of your data access quota).

In [ ]:
# Improved version of earlier query, now with aliasing & improved readability
query_improved = """
                 SELECT parent, COUNT(1) AS NumPosts
                 FROM `bigquery-public-data.hacker_news.comments`
                 GROUP BY parent
                 HAVING COUNT(1) > 10
                 """

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(query_improved, job_config=safe_config)

# API request - run the query, and convert the results to a pandas DataFrame
improved_df = query_job.to_dataframe()

# Print the first five rows of the DataFrame
improved_df.head()

## Note on using **GROUP BY**  
Note that because it tells SQL how to apply aggregate functions (like **COUNT()**), it doesn't make sense to use **GROUP BY** without an aggregate function. Similarily, if you have any **GROUP BY** clause, then all variables must be passed to either a  
1. **GROUP BY** command, or  
2. an aggregation function.  


In [ ]:
query_good = """
             SELECT parent, COUNT(id)
             FROM `bigquery-public-data.hacker_news.comments`
             GROUP BY parent
             """

Note that there are two variables: `parent` and `id`.  
* `parent` was pass to a **GROUP BY** command (in `GROUP BY parent`), and  
* `id` was paased to an aggregate function (in `COUNT(id)`)    

And this query won't work, because the `author` column isn't passed to an aggregate function, or a **GROUP BY** clause:  

In [ ]:
query_bad = """
            SELECT author, parent, COUNT(id)
            FROM `bigquery-public-data.hacker_news.comments`
            GROUP BY parent
            """

If make this error, you'll get the error message `SELECT list expression references column (column's name) which is neither grouped nor aggregated at.`

## Coding Example

In [ ]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "hacker_news" dataset
dataset_ref = client.dataset("hacker_news", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "comments" table
table_ref = dataset_ref.table("comments")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the "comments" table
client.list_rows(table, max_results=5).to_dataframe()

### Exercises
### 1) Prolific commenters  
Hacker News would like to send awards to everyone who has written more than 10,000 posts. Write a query that returns all authors with more than 10,000 posts as well as their post counts. Call the column with post counts `NumPosts`.

In case sample query is helpful, here is a query you saw in the tutorial to answer a similar question:

        query = """
                SELECT parent, COUNT(1) AS NumPosts
                FROM `bigquery-public-data.hacker_news.comments`
                GROUP BY parent
                HAVING COUNT(1) > 10
                """

In [ ]:
# Query to select prolific commenters and post counts
prolific_commenters_query = """
                            SELECT author, COUNT(1) as NumPosts
                            FROM `bigquery-public-data.hacker_news.comments`
                            GROUP BY author  
                            HAVING COUNT(1) > 10000
                            """ # Your code goes here

# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 1 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(prolific_commenters_query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
prolific_commenters = query_job.to_dataframe()

# View top few rows of results
print(prolific_commenters.head())

# Check your answer
q_1.check()

## 2) Deleted comments
How many comments have been deleted? (If a comment was deleted, the deleted column in the comments table will have the value True.)

In [ ]:
# Write your query here and figure out the answer
delete_query= """
              SELECT COUNT(1) as num_deleted_posts 
              FROM `bigquery-public-data.hacker_news.comments`
              WHERE deleted = True
              """

# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 1 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(delete_query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
deleted = query_job.to_dataframe()

# View top few rows of results
print(deleted.head())


## Reference  
https://www.kaggle.com/dansbecker/group-by-having-count   
https://www.kaggle.com/nichollette/exercise-group-by-having-count/edit  
  
---